## 스레딩으로 뭘 할 수 있지?

**스레딩은 OS에서 제공되는 기능**

**개발자가 OS에게 프로그램의 일부를 parallel 로 실행하겠다고 알리는 기능이다.**  
    이때 OS의 스케줄러가 해당 프로그램의 일부에 CPU 자원을 얼마나 할당할지 결정한다

    멀티스레드 프로그램은 멀티코어 CPU에서 각 스레드를 병렬로 실행할 수 있다. 
    운영체제는 스레드를 여러 CPU 코어에 할당하여 동시에 실행시킴으로써, 전체 작업 처리 속도를 향상시킬 수 있다.
        예를 들어 스레딩을 하면 
        하나의 프로세스 내에서 여러 스레드가 실행될 때, 이들 스레드는 동일한 주소 공간을 공유합니다. 
            이는 동일한 변수, 객체, 데이터 구조에 접근할 수 있음을 의미합니다.
            별도의 메모리 할당 없이 데이터를 공유할 수 있으므로, 
            스레드 간의 통신이 메모리 복사 없이 효율적으로 이루어질 수 있습니다.
            스레드 간에 데이터를 주고받을 때, 공유 메모리를 통해 직접 접근할 수 있게 되면
            다른 통신 방법(예: 메시지 큐, 소켓 통신 등)보다 빠르고 자원 소모가 적습니다.
        다만, 여러 스레드가 동시에 같은 데이터에 접근할 때 발생할 수 있는 동기화 문제를 해결하기 위해 
        락, 세마포어 등의 동기화 기법을 사용해야 한다.

    구체적인 스레딩 사용 예로
        CPU를 써야하는 여러 계산 위주 작업들 간에 메모리를 공유할 수 있는 다중 CPU 프로그램이 필요한 경우, 
        예를 들어 numpy는 모든 메모리를 공유하면서 여러 CPU를 사용해 행렬 연산을 가속한다
    
    스레드는 직접 관리해야 하는 저수준 도구이기 때문에
    스레딩은 대규모 병행 작업에 쓰일 때 오히려 비효율적일 수 있다.
    코드가 난해해지고, 컨텍스트 스위치가 많아진다.


## ayncio로 뭘 할수 있지?

단일 프로그램에서 동시에 여러 개의 HTTP 요청을 병행(concurrent) 실행할 수 있다.   
(requests.get()을 호출해 서버의 응답을 기다리는 작업이 완료되는 동안 다른 작업을 수행할 수 있도록 하는 등의 일)  

    CPU는 작업을 처리한 후 네트워크 Input/Output 의 완료를 기다린다.
    CPU는 네트워크 작업보다 훨씬 더 빠르므로, 
    네트워킹을 포함하는 프로그램을 실행해보면 CPU 동작 시간 중 많은 부분을 대기시간이 차지한다.

    I/O 위주 작업에 스레드 기반 병행처리보다 
    비동기 기반 병행처리를 적용할 때 이점이 있다

        1. 스레드 기반 애플리케이션에서 발생할 수 있는 특정 범주의 경합조건 등의 위험 요소가 발생하지 않는다
        (분산 마이크로 서비스 구조에서 공유 자원에 대한 프로세스 내부의 경합 같은 경합조건은 방지하지 못한다)

        2. asyncio를 통해 동시에 수천개의 소켓 연결을 간단히 처리할 수 있다


    네트워크 프로그래밍은 스레딩을 필요로 하는 영역이 아니다.
        네트워크 프로그래밍은 기다림이 많은 작업들로 구성돼 있다. 
        따라서 여러 CPU에 작업들을 효율적으로 분배하기 위한 운영체제와의 연계작업이 필요 없다. 
        또한 공유 메모리 접근 시 발생할 수 있는 경합조건과 같은 리스크를 불러오는 선점형 멀티태스킹도 필요 없다.

    속도는 asyncio로 얻을 수 있는 장점이 아니다.
        일반적으로 스레딩이 비동기 기반 병행처리보다 빠르다.
        속도가 중요하다면 Cython을 써보라.
        Cython을 사용하면 GIL 제약 (모든 스레드를 하나의 CPU에서만 처리하도록 강제)을 피할 수 있다.

    asyncio는 단일 스레드이기 때문에 GIL의 영향을 받지 않는다.
        대신 여러 CPU 코어의 장점도 얻을 수 없다.

## Chapter 3

ayncio 공식문서는 end user 개발자보다 framework/library 개발자에게 더 유용한 문서다.  
  
    ayncio 공식문서는 end user 개발자용 기능인지 framework/library 개발자용 기능인지  
    구분하지 않고 모든 API를 나열해서 보기가 힘들다.  

전체 asyncio API 중 end user 개발자에게 유용한 것들은 다음과 같다  

    asyncio 이벤트 루프 시작하기
    async/await 함수 호출하기
    루프에서 실행할 태스크 작성하기
    모든 병행 태스크 종료 후 루프 종료하기


```python
import asyncio
import time

async def f():
    print(f'{time.ctime()} hello')
    await asyncio.sleep(3)
    print(f'{time.ctime()} good bye')
    

'''
대부분의 asyncio 기반 코드는 run() 함수를 사용한다
이 함수의 역할을 이해해야 큰 애플리케이션을 잘 설계할 수 있다
'''
# asyncio.run(f()) 

def blocking():
    time.sleep(5)
    print(f'{time.ctime()} hello from a thread')

'''
아래 코드는 run() 함수 내부의 거의 모든 동작에 대한 예제다.
'''

loop = asyncio.get_event_loop() # 현재 이벤트 루프를 가져옴
    '''
    이벤트 루프(Event Loop)는 비동기 코드 실행을 관리하는 핵심 구성 요소다. 
        이벤트 루프는 다음과 같은 역할을 한다
            1. 작업 예약: 비동기 함수를 예약하고 실행 순서를 관리
            2. I/O 처리: 비동기적으로 입출력 작업을 처리
            3. 콜백 실행: 특정 이벤트가 발생했을 때 콜백 함수를 실행

    asyncio.get_event_loop()의 역할

    새로운 이벤트 루프 생성:
    현재 스레드에 이벤트 루프가 없으면 새로운 이벤트 루프를 생성하고 그것을 반환
    주로 이벤트 루프를 처음 설정할 때 사용

    asyncio.get_event_loop() 는 코루틴을 실행하기 위해 필요한 루프 인스턴스를 얻는 문법이다.
        코루틴은 비동기 함수보다 더 넓은 개념. 
            코루틴 : 실행 중간에 일시 중단하고 나중에 같은 상태에서 다시시작할 수 있는 함수

            비동기 함수는 코루틴의 일종.
            비동기 함수의 사용목적은 주로 비동기 작업을 수행하는데 있다. 
                비동기작업(네트워크 요청, 파일 I/O)

            파이썬에서는 async 키워드를 사용해 비동기함수를 정의

            async 키워드로 정의된 비동기함수는
            await 키워드를 사용해 다른 비동기 함수나 비동기 작업을 호출하고
            그 작업이 완료되기를 기다린다

    asyncio.get_event_loop()를 동일 스레드에서 호출하면 
    코드의 어느 부분에서 호출하든 매번 똑같은 루프 인스턴스를 반환한다. 

    그런데 async def 함수 내에서 호출하는 경우에는 asyncio.get_running_loop()를 호출해야 한다. 
    asyncio.get_running_loop()를 호출하면 현재 실행 중인 이벤트 루프를 반환한다.
    실행 중인 이벤트 루프가 없으면 RuntimeError를 발생 시킨다
    '''

task = loop.create_task(f()) # : 비동기 함수(코루틴) f를 이벤트 루프에서 실행하도록 예약하고 task 객체로 반환. 
# create_task()에 전달되는 함수는 반드시 async def로 정의된 비동기 함수여야 한다.

loop.run_in_executor(None, blocking) # 기본 스레드 풀에서 blocking 함수가 실행되도록 예약. 
# run_in_executor는 이벤트 루프가 시작될 때(run_until_complete가 실행되어야) 실행된다. 
# run_in_executor() 에 들어가는 함수는 일반적으로
# 라이브러리에서 가져다 쓰는 "동기함수"이거나, CPU 바운드 작업을 위주로 해야하는 "동기함수" 또는 블로킹 I/O 작업을 하는 "동기함수"다. 
# create_task()에 들어가는 함수는 주로 I/O 바운드의 비동기함수다. 
# 즉 run_in_executor를 쓰는 이유는 
# 동기함수를 별도의 스레드나 프로세스에서 실행하여, ayncio가 돌아가는 메인 이벤트 루프가 블로킹되지 않도록 하기위해 쓴다.
# 즉 run_in_executor를 쓰게 됨으로써 멀티스레딩 또는 멀티프로세싱을 하게 되는 것이다.
'''
멀티태스킹이 잘 되기 위해서는 비동기와 동기 함수를 적절히 어우러지게 사용해야 한다.
이는 await 키워드를 사용해 이벤트 루프에게 제어권을 넘기는 식으로 이뤄진다. 
await 키워드는 현재 작업을 일시 중단하고, 이벤트 루프가 다른 작업을 실행할 수 있도록 한다. 

동기함수는 호출이 완료될때까지 제어권을 다른 작업에 넘기지 않는다. 
동기함수는 비동기 함수내에서 호출될 수 있고 이는 비동기 작업을 블로킹할 수 있다. 
비동기 함수 내에 동기함수를 호출해야 할 경우, 
동기함수를 run_in_executor에 넣고 await 키워드를 사용하면, 
이벤트 루프가 블로킹되지 않으면서 동기함수를 실행할 수 있다.

따라서 run_in_executor()는 메인 이벤트 루프를 블로킹하지 않는다. 
단지 작업을 실행할 스레드 풀이나 프로세스 풀에 작업을 스케줄링한다.
run_in_executor를 호출하면 Future개체를 반환받는다. 
익스큐터에 할당된 작업은 asyncio 이벤트 루프가 실행 중일 때 시작된다.
run_until_complete()를 호출하면 이벤트 루프가 실행되어 할당된 작업들이 시작될 수 있다.
'''

# 그렇다면 동기함수를 비동기함수로 바꿔 create_task에 넣으면 안되나?
# 동기함수를 비동기 함수로 변환할 수 없는 경우가 있다. 
# 기존 라이브러리에서 제공되는 함수를 그대로 써야 하거나, CPU 집약적인 작업을 해야 하는 경우가 그렇다.
# 전자는 라이브러리 함수를 가져다 쓰는 경우이므로 비동기함수로 바꿀 수 없고 후자는 비동기 함수로 바꾸는 게 의미가 없다. 
# 물론 동기적으로 설계된 라이브러리 대신 비동기적으로 설계된 라이브러리를 쓸수도 있다.
# 그러나 requests 대신 쓸 수 있는 aiohttp 라이브러리처럼 비동기 API를 제공하는 경우는 흔치 않다.
# with open 함수처럼 많은 파일 시스템 API는 동기적으로 동작한다.
# 일부 비동기 파일 시스템 API가 있지만, 모든 파일 시스템이나 운영 체제가 이를 지원하는 것은 아니다.
# sqlite3 처럼 DB에 쿼리를 날리는 함수 같은 블로킹 I/O작업을 하는 함수들도 비동기 함수로 변환하기 어렵다.
# 블로킹 호출의 본질적인 특성, 성능 안정성 문제, 복잡한 상호작용 때문이다. 
# 기존의 동기적 코드를 비동기 코드로 변환하는 것은 성능과 안정성 문제를 야기할 수 있다. 
# 동기 코드가 잘 작동하고 있는 경우, 이를 비동기 코드로 변환하는 것은 코드 복잡성을 증가시키고, 예기치 않은 버그를 초래할 수 있다
# 위와 같은 이유로 동기 함수는 run_in_executor를 사용하여 멀티스레딩이나 멀티프로세싱으로 처리하고, 
# 비동기 함수로 변환할 수 있는 작업은 create_task를 사용하여 이벤트 루프에서 비동기적으로 실행하는 것이다.


loop.run_until_complete(task) # 호출하면 현재 메인 스레드를 블로킹하고 메인스레드가 이벤트 루프를 실행하게 한다
# 비동기 task가 완료될 때까지 이벤트 루프 실행한다.  
# 이벤트 루프는 예약된 모든 작업을 실행한다. 비동기 함수 f가 비동기적으로 실행됨. 동시에 스레드 풀에서 blocking 함수도 실행됨. 

'''
이하 코드들은 비동기 작업을 깔끔하게 끝내기 위해 사용된다. 비동기 작업을 정리하고, 이벤트 루프를 안전하게 종료하기 위한 목적을 가지고 있습니다.
'''
pending = asyncio.all_tasks(loop=loop) # 현재 이벤트 루프에서 실행 중이거나 대기 중인 모든 작업(task)을 가져옴. 
# pending 변수 내의 태스크 목록에는 run_in_executor 호출시 매개변수로 전달했던 blocking 에 대한 호출은 포함하지 않는다. 
# all_tasks()는 Task만 반환하고 Future는 반환하지 않는다 

for task in pending:
    task.cancel() # 각 작업을 취소.

group = asyncio.gather(*pending, return_exceptions = True) # 모든 남은 작업을 하나로 모아서 처리, 예외 발생 시 반환.

loop.run_until_complete(group) # 그룹 작업이 완료될 때까지 이벤트 루프를 실행.

loop.close() # 이 함수는 루프의 모든 대기열을 비우고 익스큐터를 종료시킨다. 
# 이 함수는 정지된 루프에 대해 호출해야한다. 
# 정지된 루프는 다시 실행될 수 있으나 닫힌 루프는 완전히 끝난 것이다. 
# 정확한 종료처리를 위해서는 더 많은 보강이 필요하다.
```

## 3.2 asyncio의 탑

네트워크 애플리케이션 개발 중 asyncio 라이브러리를 사용하는 경우  
end user 개발자에게 중요한 계층(API)은 다음과 같다.  

#### 계층1 async def, async with, async for, await
    async def 함수를 작성하는 방법  
    다른 코루틴을 호출하고 실행하기 위해 await를 사용하는 방법  

#### 계층2 asyncio.run()
    이벤트 루프를 시작하고, 종료하고, 상호작용하는 방법  

#### 계층5 run_in_executor()
    익스큐터를 활용하는 방법  
    그런데 대부분의 서드파티 라이브러리는 ayncio와 호환되지 않는다  
    예를 들어 SQLAlchemy의 ORM 라이브러리는 asyncio와 호환되는 기능을 지원하지 않는다  

#### 계층6 Asyncio.Queue
    한 개 이상의, 긴 시간동안 실행하는 코루틴에 데이터를 전달할 때는  
    asyncio.Queue 를 쓰는게 가장 적합한 방법이다.  
    스레드에 데이터를 배분할 때 queue.Queue를 쓰는 것과 동일하다  

#### 계층9 StreamReader, StreamWriter
    Streams API는 소켓 통신을 처리하는 가장 간단한 방법이다.  
    asyncio와 호환되는 소켓 통신용 서드파티 라이브러리(aiohttp)를 사용한다면 Streams API를 쓸 필요는 없다.  


이제 아래 계층 부터 차례차례 사용법을 알아본다

## 3.3 코루틴

### 코루틴은 완료되지 않은 채 일시정지했던 함수를 재개할 수 있는 기능을 가진 객체다

3.4에서 aysncio가 추가되고
3.5에서 async와 await가 도입되기 전  
asyncio는 제너레이터로 코루틴을 구현했다.  
(@asyncio.coroutine 로 처리된 제너레이터 함수 내에 yield from 구문이 들어있는 형식)  
  
async def를 사용한 코루틴은 네이티브 코루틴이라고 불린다  

async def 함수와 이 함수가 반환하는 코루틴은 제너레이터와 흡사한 방식으로 동작한다  
실제로 코루틴이 반환할 때 StopIteration예외가 발생한다


용어 정리  
**비동기 함수 (Asynchronous Function):**   
async def로 정의된 함수. 호출되면 실행되지 않고 코루틴 객체를 반환합니다.  
  
**코루틴 (Coroutine):**  
비동기 함수를 호출하여 반환된 객체.   
이 객체는 await 키워드로 실행을 중단하고, 재개할 수 있습니다.  

In [1]:
import inspect

def g():
    yield 123

gen = g()
gen

print(type(g))   # g는 제너레이터가 아니다. 그냥 함수다.
print(type(gen)) # 제러네이터는 g를 호출하여 값을 반환받아야 한다.
print(next(gen))

async def f():
    return 123

print(type(f))   # 코루틴도 동일하다. f는 코루틴이 아니라 그냥 함수다. 
print(inspect.iscoroutinefunction(f)) # 정확히 말하면 코루틴 함수다. (비동기 함수)

coro = f() # async def 함수를 호출하여 코루틴 객체를 받아와야 한다.
print(type(coro)) # 코루틴 함수를 호출하여 반환받은 값이 코루틴이다

<class 'function'>
<class 'generator'>
123
<class 'function'>
True
<class 'coroutine'>


중요한 것은 파이썬의 코루틴들 사이에서 실행을 전환하는 방식이다

In [3]:
async def f(): # f는 코루틴 함수다.
    return 123

coro = f() # 코루틴 함수를 호출하면 코루틴 객체가 반환된다.
# 정확히는 f를 호출해보 함수 본체는 실행되지 않는다
# 함수 본체를 실행하려면 코루틴 객체를 실행해야 한다
# 코루틴 객체를 실행하는 방법은 아래와 같다.
try:
    coro.send(None) # 코루틴 객체에 None을 전달하여 초기화 한다. 
    # 이 호출은 코루틴을 시작하고 함수 본체가 실행된다
    # 코루틴이 시작되면 await 를 만날때까지 코루틴을 실행한다.

    # 이벤트 루프는 내부적으로 동일한 방식을 통해 코루틴에 대해 초기화를 진행한다.
    # loop.create_task(coro) 혹은 await coro를 통해 실행하면 
    # loop가 알아서 .send(None)을 내부적으로 실행한다
except StopIteration as e: # 코루틴이 반환하면 StopIteration 예외가 발생한다.
    # 코루틴 함수는 123을 즉시 리턴하므로 코루틴은 곧바로 StopIteration 예외를 발생시키며 종료된다
    # 예외의 value 속성을 확인해 코루틴의 반환값을 확인할 수 있다
    print(f'the answer was: {e.value}')

the answer was: 123


In [4]:
'''
send()와 StopIteration 두 지점이 각각 코루틴 실행의 시작과 끝이다
이 내부 동작에 대해 자세히 알 필요는 없다
이벤트 루프가 내부동작을 처리해 코루틴의 실행을 제어한다
end user 개발자 입장에서는 
단순히 루프에서 코루틴이 실행되도록 스케줄링 하면
일반적인 함수처럼 탑다운 형태로 실행될 것이다
'''
async def f():
    return 123

coro = f()
coro.send(None)


StopIteration: 123

## 3.3.2 await 키워드

### await 키워드는 항상 매개변수 하나가 필요하다.  
다음 중 하나여야 한다.  
- 코루틴(즉 async def 함수의 반환값)  
- __await__() 라는 특별 메서드를 구현한 모든 객체, 이 메서드는 반드시 이터레이터를 반환해야 한다  
  
책에서는 첫번째 경우, 즉 매개변수로 코루틴을 취하는 경우만 다룬다  
두번째 경우는 일상적인 asyncio 프로그래밍에서 사용할 일이 거의 없다  

In [ ]:
import asyncio

async def f(): # f()를 호출하면 코루틴을 반환한다. 이는 f()에 대해 await 할 수 있다는 뜻이다.
    await asyncio.sleep(3)
    return 123

async def main():
    result = await f() # f()가 완료되면 result 변수의 값은 123이 될 것이다.
    return result

#### 코루틴에 예외를 주입하는 방법 = 가장 일반적인 취소방법
task.cancel()을 호출하면, 이벤트 루프는 내부적으로 coro.throw()를 사용해  
코루틴 내부에서 asyncio.CancelledError 예외를 발생시킨다

In [8]:
import asyncio

async def f():
    await asyncio.sleep(0)
    return 123

coro = f()
coro.send(None)
# await를 만나기 전까지 코루틴이 실행되므로 중단된다.

In [10]:
import asyncio

async def f():
    await asyncio.sleep(0)
    return 123

coro = f()
coro.send(None)
coro.send(None)  
# 실행이 멈춘 코루틴을 다시 재개하면 즉시 123을 반환하고 StopIteration예외를 발생시킨다

StopIteration: 123

In [9]:
import asyncio

async def f():
    await asyncio.sleep(0)
    return 123

coro = f()
coro.send(None)
coro.throw(Exception, 'blah') # 또 다른 send를 호출하지 않고
# throw()를 호출하여 예외 클래스와 값을 전달하면 
# 이를 통해 await 지점에서 코루틴 내에 예외를 발생시킨다

Exception: blah

In [11]:
import asyncio

async def f():
    try:
        while True: await asyncio.sleep(0)
    except asyncio.CancelledError: # asyncio 라이브러리에서는 태스크 취소를 위해
        # 특정 예외 형을 사용한다. 그것이 asyncio.CancelledError 다.
        print('cancelled')
    
coro = f()
coro.send(None)
coro.send(None)
coro.throw(asyncio.CancelledError) # 코루틴 외부에서 코루틴에 예외를 주입한다는 점에 유의하자.
# 즉 이벤트 루프에 의해 send()와 throw()를 직접 실행한다.
# StopIteration 예외가 코루틴이 종료하는 일반적인 방법이므로 태스크 취소시 코루틴은 종료된다


cancelled


StopIteration: 

In [9]:
import asyncio

async def f():
    await asyncio.sleep(0)
    return 123

loop = asyncio.get_running_loop()  
coro = f()
loop.run_until_complete(coro)


RuntimeError: This event loop is already running

## 3.4 이벤트 루프

지금까지 코루틴의 비동기 처리에 대해 확인해보았다.  
실제 asyncio 의 이벤트 루프는 코루틴 간 전환, StopIteration 예외처리, 소켓과 파일 디스크립터의 이벤트 수신 승도 처리하므로 다소 차이가 있다.  
  
이벤트 루프를 직접 다루지 않아도 await와 asyncio.run(coro) 만으로도 asyncio를 사용할 수 있다.  
하지만 경우에 따라 이벤트 루프를 다뤄야 하는 경우가 있다.  
  
이벤트 루프를 얻는 두가지 방법  
  
추천 : asyncio.get_running_loop() 코루틴 내에서 호출 가능  
비추천 : asyncio.get_event_loop() 어디서든 호출 가능  

In [11]:
loop = asyncio.get_event_loop()
loop2 = asyncio.get_event_loop()
loop is loop2 # loop와 loop2는 동일한 인스턴스를 참조한다

# 코루틴 함수 내에서 루프 인스턴스에 접근하기 위해서는
# get_event_loop() 또는 get_running_loop()를 호출하면된다

True

In [12]:
loop = asyncio.get_running_loop()
loop2 = asyncio.get_running_loop()
loop is loop2

True

In [13]:
loop = asyncio.get_event_loop()
loop2 = asyncio.get_running_loop()
loop is loop2

True

### get_running_loop() 와 get_event_loop() 의 차이
get_event_loop() 메서드는 동일한 스레드 내에서만 동작하는 목적으로 만들어졌다.


## 3.5 Task와 Future

Future 클래스는 Task의 상위 클래스로 루프와 관련된 모든 기능을 제공한다  
  
Future는 어떤 동작의 미래에 일어날 완료상태를 나타내고 루프에 의해 관리된다  
Task는 Future와 완전히 동일한 기능에 더하여, 동작을 async def로 정의한 함수와  
create_task() 함수를 통해 생성한 코루틴으로 지정한다  
  
end user 개발자는 Task를 사용하는 편이 더 일반적이다.  
Future를 사용해야 하는 경우는 익스큐터를 사용할 때다.  
익스큐터를 사용할 떄는 Task가 아닌 Future인스턴스를 반환받는다  

In [1]:
import asyncio

async def main(f: asyncio.Future):
    await asyncio.sleep(1)
    f.set_result(("I Have finished"))

loop = asyncio.get_event_loop()

future = asyncio.Future() # 이 인스턴스는 기본적으로 loop에 연결되지만
# 어떠한 코루틴에도 연결되지 않았고 연결되지도 않는다
print(future.done()) # future인스턴스에는 done이라는 메서드가 있어 상태를 확인할 수 있다

False


In [2]:
loop.create_task(main(future)) # main() 코루틴을 스케쥴링한다. 
# 동시에 future 인스턴스르 매개변수로 전달한다
# main 코루틴이 하는 일은 잠깐 자고 일어나서 futue 인스턴스의 상태를 변경하는 것이다
loop.run_until_complete(future) # task 인스턴스가 아닌 future 인스턴스에 대해 run_until_complete를 사용한다
print(future.done())
print(future.result())

RuntimeError: This event loop is already running

첫 번째 코드:
```python
import asyncio

async def main(f: asyncio.Future):
    await asyncio.sleep(1)
    f.set_result(("I Have finished"))

loop = asyncio.get_event_loop()
future = asyncio.Future() 
fut = loop.create_task(main(future)) 
loop.run_until_complete(fut) 
print(future.done())
print(future.result())
```

두 번째 코드:
```python
import asyncio

async def main(f: asyncio.Future):
    await asyncio.sleep(1)
    f.set_result(("I Have finished"))

loop = asyncio.get_event_loop()
future = asyncio.Future() 
loop.create_task(main(future)) 
loop.run_until_complete(future) 
print(future.done())
print(future.result())
```


    두 코드의 차이점:
    1. 첫 번째 코드에서는 `loop.run_until_complete(fut)`를 호출합니다. 
    여기서 `fut`는 `loop.create_task(main(future))`로 생성된 Task 객체입니다.

    2. 두 번째 코드에서는 `loop.run_until_complete(future)`를 호출합니다.
    여기서 `future`는 직접 생성된 `asyncio.Future` 객체입니다.

    이 차이점은 코드의 실행 방식에 영향을 미칩니다. 자세히 설명하겠습니다:

    - 첫 번째 코드에서 `loop.run_until_complete(fut)`를 호출하면, 
    `fut`가 완료될 때까지 이벤트 루프가 실행됩니다. 
    `fut`는 `main` 코루틴의 Task 객체이므로 `main`이 끝날 때까지 대기하게 됩니다. 
    `main`이 끝나면 `future`의 결과가 설정됩니다.

    - 두 번째 코드에서 `loop.run_until_complete(future)`를 호출하면, 
    `future`가 완료될 때까지 이벤트 루프가 실행됩니다. 
    `main` 코루틴이 `future`의 결과를 설정하기 때문에, 
    사실상 두 코드 모두 `main` 코루틴의 완료를 기다리게 됩니다.

    따라서, 결과적으로 두 코드 모두 같은 동작을 합니다. 
    하지만 첫 번째 코드에서 명시적으로 `fut`를 `run_until_complete`에 전달하는 것이 더 직관적입니다.
    Task 객체 `fut`가 완료될 때까지 대기한다는 의도를 명확히 전달하기 때문입니다.

## 3.6 async with

#### 컨텍스트 관리자 (Context Manager)가 뭐지?
컨테스트 관리자는 리소스를 할당하고 해제하는 과정을 단순화하는 데 사용되는 파이썬의 객체입니다.   
  
주로 파일, 네트워크 연결, 락, 데이터베이스 연결 등의 리소스를 다룰 때 유용합니다.   
  
with 문과 함께 사용되어, 특정 블록의 실행 전후에 설정과 정리를 자동으로 처리합니다.  

#### 컨텍스트 관리자 (Context Manager)가 왜 필요하지?

리소스 관리:  

    리소스를 열고 닫는 과정을 자동화하여 코드의 안정성과 가독성을 높입니다.  
  
예외 처리:  

    예외가 발생하더라도 리소스를 적절하게 해제할 수 있습니다.  
  
사용 방법: 

    파일 처리: 파일을 열고 자동으로 닫는 데 사용됩니다.  
```python
# 파일 처리 예제
with open('file.txt', 'r') as file:
    data = file.read()
# 이 블록이 끝나면 파일이 자동으로 닫힘
```
    락: 멀티스레딩 환경에서 락을 자동으로 획득하고 해제하는 데 사용됩니다.  
    
    데이터베이스 연결: 연결을 열고 닫는 과정을 자동화합니다.  

#### 컨텍스트 관리자를 썼을 때와 안썼을 때의 차이

컨텍스트 관리자를 사용하지 않은 경우    

    컨텍스트 관리자를 사용하지 않으면 파일을 열고 닫는 작업을 명시적으로 수행해야 합니다.  
    만약 예외가 발생하면 파일이 제대로 닫히지 않을 수 있습니다.  

```python
def read_file(file_path):
    file = open(file_path, 'r')  # 파일 열기
    try:
        data = file.read()  # 파일 읽기
        return data
    finally:
        file.close()  # 파일 닫기

try:
    content = read_file('example.txt')
    print(content)
except Exception as e:
    print(f"Error: {e}")
# 이 코드에서는 파일을 열고 닫는 작업을 수동으로 관리해야 합니다. 
#  예외가 발생하더라도 finally 블록에서 파일을 닫도록 해야 합니다.
```

컨텍스트 관리자를 사용한 경우  

    컨텍스트 관리자를 사용하면 with 문이 파일을 자동으로 닫아주기 때문에 코드가 간결하고 안전합니다.

```python
def read_file(file_path):
    with open(file_path, 'r') as file:  # 파일 열기 및 자동 닫기
        data = file.read()  # 파일 읽기
        return data

try:
    content = read_file('example.txt')
    print(content)
except Exception as e:
    print(f"Error: {e}")


```


#### 블로킹 방식
이 컨텍스트 관리자는 전용스레드에서 사용해야 한다.  
그렇지 않으면 이 함수가 끝날 때까지 전체 프로그렘이 일시 중지될 것이다   

In [ ]:
from contextlib import contextmanager

@contextmanager # 데커레이터는 제너레이터 함수를 동기 컨텍스트 관리자로 변환한다
def web_page(url):
    data = download_webpage(url) #  download_webpage 함수는 내부적으로 블로킹 I/O가 있다고 하자
    yield data 
    update_stats(url) # update_stats 함수도 내부적으로 블로킹 I/O가 있다고 하자

with web_page('google.com') as data:
    process(data) # process 함수는 빠르고 CPU bound의 논블로킹 함수로 가정한다

#### 논블로킹 방식
비동기 방식으로 블로킹을 피할 수 있다

In [ ]:
from contextlib import asynccontextmanager

@asynccontextmanager
async def web_page(url): # web_page 함수는 비동기 제너레이터 함수가 된다.
    # 이 함수를 호출하면 비동기 제너레이터를 반환한다
    data = await download_webpage(url) #  download_webpage 함수는 코루틴을 반환한다고 가정하자
    yield data 
    await update_stats(url) # update_stats 함수는 코루틴을 반환한다고 가정하다

async with web_page('google.com') as data:
    process(data) # process 함수는 빠르고 CPU bound의 논블로킹 함수로 가정한다

위 예제에서 2개의 블로킹 함수인 download_wabpage()와 update_stats()를  
비동기함수로 수정할 수 없다고 하자.  
즉 코루틴을 반환하도록 수정할 수 없다는 것이다.  
이 문제를 해결하는 방법은 별도의 스레드에서 익스큐터로 블로킹 호출을 하는 것이었다

In [ ]:
# 기존 동기 함수를 비동기 코드와 함께 사용할 수 있다.
from contextlib import asynccontextmanager

@asynccontextmanager
async def web_page(url):
    loop = asyncio.get_event_loop()
    data = await loop.run_in_executor(None, download_webpage, url)
    yield data
    await loop.run_in_executor(None, update_stats, url)

async with web_page('google.com') as data:
    process(data)

## 3.7 async for
비동기 이터레이터

In [5]:
# 비동기가 아닌 전통적인 이터레이터
class A:
    def __iter__(self): 
        self.x = 0
        return self 
    def __next__(self): # 이 __next__ 메서드를 aysnc def 코루틴 함수로 만들어
        # await 를 걸 수 있게 한 방식이 async for가 동작하는 방식이다.
        if self.x > 2:
            raise StopIteration
        else:
            self.x += 1
            return self.x

for i in A():
    print(i)

1
2
3


#### 예제
redis 데이터베이스에서 여러 개의 키에 대해 반복을 수행해 데이터를 확인할 때  
각 데이터를 요청 시점에 가져오려 한다고 하자  

In [6]:
import asyncio
from aioredis import create_redis 
# 레디스 DB와 연결하기 위해 aioredis 라이브러리의 고수준 인터페이스를 사용한다

async def main():
    redis = await create_redis(('localhost', 6379))
    # aioredis 라이브러리를 사용하여 Redis 서버에 비동기적으로 연결합니다. 
    # Redis 서버는 로컬 호스트의 6379 포트에서 실행 중이라고 가정합니다.
    keys = ['Asia', 'Africa', 'Americas', 'Europe'] 
    # 키에 연관된 값의 개수가 대량이고 모두 레디스에 저장돼있다고 가정하자

    async for value in OneAtATime(redis, keys):
        # async for를 사용하면, 반복 중에 반복 자체를 일시정지 할 수 있다
        await do_something_with(value)
        # 레디스에서 얻은 데이터에 대해 데이터를 변환해 다른 돗에 전달하는 I/O 위주 동작을 수행한다고 가정하자

class OneAtATime:
    def __init__(self, redis, keys):
        self.redis = redis
        self.keys = keys
    def __aiter__(self): # async for를 사용하려면 _iter__를 구현해야 한다
        self.ikeys = iter(self.keys) # self.keys의 이터레이터인 ikeys를 만들고 
        return self # self를 반환하여 이터레이터 자체를 반환
    # 이로 인해 async for 문은 OneAtATime 객체를 비동기 이터레이터로 인식합니다.
    async def __anext__(self): # # __aiter__() 메서드는 def지만
        # __anext__ 메서드는 async def로 선언한다
        # 비동기 이터레이터의 다음 값을 반환하기 위해 정의된 메서드
        # async for 문이 각 반복을 시작할 때마다 __anext__ 메서드가 호출됩니다.
        try:
            k = next(self.ikeys)
        except StopIteration: # self.ikeys를 소모해 StopIteration예외가 발생하면 
            # StopAsyncIteration 로 전환한다. 이것이 비동기 이터레이터 내에서
            # 정지신호를 발생시키는 방법이다
            raise StopAsyncIteration
    
        value = await self.redis.get(k) # 데이터를 가져오는 작업이 완료될때까지 await하여
        # 이벤트 루프에서 다른 동작이 수행되도록 할 수 있다.
        return value

asyncio.run(main())

ModuleNotFoundError: No module named 'aioredis'

위 코드를 비동기 제너레이터로 더 간단히 만들 수 있다

In [ ]:
import asyncio
from aioredis import create_redis 
# 레디스 DB와 연결하기 위해 aioredis 라이브러리의 고수준 인터페이스를 사용한다

async def main():
    redis = await create_redis(('localhost', 6379))
    # aioredis 라이브러리를 사용하여 Redis 서버에 비동기적으로 연결합니다. 
    # Redis 서버는 로컬 호스트의 6379 포트에서 실행 중이라고 가정합니다.
    keys = ['Asia', 'Africa', 'Americas', 'Europe'] 
    # 키에 연관된 값의 개수가 대량이고 모두 레디스에 저장돼있다고 가정하자

    async for value in one_at_a_time(redis, keys):
        # async for를 사용하면, 반복 중에 반복 자체를 일시정지 할 수 있다
        await do_something_with(value)
        # 레디스에서 얻은 데이터에 대해 데이터를 변환해 다른 돗에 전달하는 I/O 위주 동작을 수행한다고 가정하자

async def one_at_a_time(redis, keys):
    for k in keys:
        value = redis.get(k)
        yield value

asyncio.run(main())

## 3.9 비동기 컴프리헨션

In [7]:
import asyncio

async def f(x):
    await asyncio.sleep(0.1)
    return x + 100

async def factory(n):
    for x in range(n):
        await asyncio.sleep(0.1)
        yield f, x

async def main():
    results = [await f(x) async for f, x in factory(3)]
    print('results = ', results)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

## 3.10 시작과 종료

대부분의 비동기 기반 프로그램은 수명이 긴 네트워크 기반 애플리케이션이다  
이런 종류의 프로그램은 올바르게 시작 start up 하고 종료 shutdown 하려면 상당히 복잡한 처리가 필요하다  

asyncio 애플리케이션을 시작하는 일반적인 방법은  
main() 코루틴 함수를 정의하고  
asyncio.run()으로 실행하는 것이다.  
  

async def main() 함수 종료시 asyncio.run() 내에서 실행되는 동작은 다음과 같다  

    1. 아직 보류중인 태스크 객체를 모두 수집한다
    2. 이 태스크들을 모두 취소한다
    취소하면 각각의 실행 중인 코루틴 내에서 CancelledError가 발생하는데
    코루틴 함수의 본문에서 try/except로 처리할지 말지 선택할수 있다
    3. 태스크 모두를 그룹 태스크로 수집한다
    4. 그룹 태스크에 대해 run_until_complete()를 사용해 모든 태스크가 완료되기까지 대기한다
    종료되기까지 대기한다는 의미는 대기중인 태스크에 발생한 CancelledError 예외가 처리되기까지 대기한다는 의미다

In [ ]:
# Example B-1. Cutlery management using asyncio
import sys
import asyncio
from attr import attrs, attrib


# Instead of a ThreadBot, we now have a CoroBot. 
# This code sample uses only one thread, 
# and that thread will be managing all 10 separate CoroBot object instances
# one for each table in the restaurant.
class CoroBot():
    def __init__(self):
        self.cutlery = Cutlery(knives=0, forks=0)
        # Instead of queue.Queue, we’re using the asyncio -enabled queue.
        self.tasks = asyncio.Queue()

    async def manage_table(self):
        while True:
            task = await self.tasks.get() # 여기가 중요한 부분이다
            # 서로 다른 CoroBot 인스턴스 간에 컨텍스트 전환을 할 수 있는 유일한 위치는
            # 바로 await 키워드가 있는 곳이다.
            # 이 함수의 나머지 부분에서는 컨텍스트 전환이 일어날 수 없다
            # 이로 인해 주방 식기 재고를 수정할 때 Race Condition이 발생하지 않는다
            # await 키워드가 있는 곳에서만 컨텍스트 전환이 일어나므로 관측가능하다
            # 이를 통해 병행 애플리케이션에서 경합조건의 가능성을 훨씬 쉽게 유추할 수 있다
            if task == 'prepare table':
                kitchen.give(to=self.cutlery, knives=4, forks=4)
            elif task == 'clear table':
                self.cutlery.give(to=kitchen, knives=4, forks=4)
            elif task == 'shutdown':
                return


@attrs
class Cutlery:
    knives = attrib(default=0)
    forks = attrib(default=0)

    def give(self, to: 'Cutlery', knives=0, forks=0):
        self.change(-knives, -forks)
        to.change(knives, forks)

    def change(self, knives, forks):
        self.knives += knives
        self.forks += forks


kitchen = Cutlery(knives=100, forks=100)
bots = [CoroBot() for i in range(10)]

for b in bots:
    for i in range(int(sys.argv[1])):
        b.tasks.put_nowait('prepare table')
        b.tasks.put_nowait('clear table')
    b.tasks.put_nowait('shutdown')
print('Kitchen inventory before service:', kitchen)

loop = asyncio.get_event_loop()
tasks = []
for b in bots:
    t = loop.create_task(b.manage_table())
    tasks.append(t)

task_group = asyncio.gather(*tasks)
loop.run_until_complete(task_group)
print('Kitchen inventory after service:', kitchen)

# 터미널 실행 코드
# python cutlery_test_coroboy.py 100000